In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import kagglehub
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split
import math
from tensorflow.keras.saving import register_keras_serializable
from sklearn.metrics import accuracy_score, confusion_matrix
from datetime import datetime
from augmentations import generate_dataset_with_augmentation




tf.random.set_seed(42)
np.random.seed(42)
# Check for GPU availability
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Download the ReducedMNIST dataset
path = kagglehub.dataset_download("mohamedgamal07/reduced-mnist")
print(f"Dataset downloaded to: {path}")

# List the files in the dataset
files = os.listdir(path)
print("Files in the dataset:", files)

path = path+"/Reduced MNIST Data"

train_dir = path +'/Reduced Trainging data'
test_dir = path+'/Reduced Testing data'

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize the images to the range [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

num_classes = 10
IMG_SIZE = 28
NOISE_DIM = 100
BATCH_SIZE = 128
EPOCHS = 75

# Reshape the images to include the channel dimension (28, 28, 1)
x_train = x_train.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
x_test = x_test.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

# Print dataset shapes
print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}, y_test shape: {y_test.shape}")



TensorFlow version: 2.19.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Dataset downloaded to: /home/youssef-abuzeid/.cache/kagglehub/datasets/mohamedgamal07/reduced-mnist/versions/1
Files in the dataset: ['Reduced MNIST Data']


In [2]:
data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train_gen = data_gen.flow_from_directory(train_dir,
                                          target_size=(IMG_SIZE, IMG_SIZE),
                                          color_mode='grayscale',
                                          class_mode='sparse',
                                          batch_size=BATCH_SIZE,
                                          shuffle=True)
test_gen = data_gen.flow_from_directory(test_dir,
                                            target_size=(IMG_SIZE, IMG_SIZE),
                                            color_mode='grayscale',
                                            class_mode='sparse',
                                            batch_size=BATCH_SIZE,
                                            shuffle=False)

                                            
                                          

Found 10000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


## Define the GAN Architecture


### First we'll define the Generator

In [3]:
@register_keras_serializable()
class Generator(tf.keras.Model):
    def __init__(self, noise_dim=NOISE_DIM, num_classes=10, **kwargs):
        super(Generator, self).__init__(**kwargs)
        self.noise_dim = noise_dim
        self.num_classes = num_classes
        
        # Increase label embedding dimension
        self.label_embedding = tf.keras.layers.Embedding(num_classes, 100)  # Increased from 50
        self.label_dense = tf.keras.layers.Dense(100, use_bias=False)       # Increased from 50
        
        # Combined noise and label processing
        self.combined_dense = tf.keras.layers.Dense(7*7*256, use_bias=False)
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.relu1 = tf.keras.layers.ReLU()
        self.dropout = tf.keras.layers.Dropout(0.3)  # Added dropout layer

        
        # Create label condition vectors for each layer
        self.cond1 = tf.keras.layers.Dense(128, use_bias=False)  # For first upsampling block
        self.cond2 = tf.keras.layers.Dense(64, use_bias=False)   # For second upsampling block
        
        # Reshape layer
        self.reshape = tf.keras.layers.Reshape((7, 7, 256))
        
        # First upsampling block
        self.conv_transpose1 = tf.keras.layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False)
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.relu2 = tf.keras.layers.ReLU()
        self.dropout2 = tf.keras.layers.Dropout(0.3)  # Added dropout layer
        # Second upsampling block
        self.conv_transpose2 = tf.keras.layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False)
        self.bn3 = tf.keras.layers.BatchNormalization()
        self.relu3 = tf.keras.layers.ReLU()
        self.dropout3 = tf.keras.layers.Dropout(0.3)  # Added dropout layer
        # Output layer with tanh activation
        self.output_layer = tf.keras.layers.Conv2D(1, (5, 5), padding='same', use_bias=True, activation='tanh')
        
        # Define conditioning layers for each block
        self.gamma1_layer = tf.keras.layers.Dense(128)  # For first block conditioning
        self.beta1_layer = tf.keras.layers.Dense(128)
        self.gamma2_layer = tf.keras.layers.Dense(64)   # For second block conditioning
        self.beta2_layer = tf.keras.layers.Dense(64)

    def call(self, inputs, training=False):
        noise, labels = inputs
        
        # Process labels - enhanced embedding
        label_embedding = self.label_embedding(labels)
        label_embedding = tf.keras.layers.Flatten()(label_embedding)
        label_info = self.label_dense(label_embedding)
        
        # Prepare conditional vectors for each layer
        cond1_info = self.cond1(label_embedding)  # For first block
        cond2_info = self.cond2(label_embedding)  # For second block
        
        # Concatenate noise and label info
        x = tf.concat([noise, label_info], axis=1)
        
        # Process combined input
        x = self.combined_dense(x)
        x = self.bn1(x, training=training)
        x = self.relu1(x)
        x = self.dropout(x, training=training)  # Apply dropout
        # Reshape
        x = self.reshape(x)
        
        # First upsampling block with conditioning
        x = self.conv_transpose1(x)
        # Apply conditional batch normalization with predefined layers
        x = self.conditional_bn(x, cond1_info, training, self.bn2, self.gamma1_layer, self.beta1_layer)
        x = self.relu2(x)
        x = self.dropout2(x, training=training)
        
        # Second upsampling block with conditioning
        x = self.conv_transpose2(x)
        # Apply conditional batch normalization with predefined layers
        x = self.conditional_bn(x, cond2_info, training, self.bn3, self.gamma2_layer, self.beta2_layer)
        x = self.relu3(x)
        x = self.dropout3(x, training=training)
        # Output layer
        x = self.output_layer(x)
        
        return x
    
    def conditional_bn(self, x, condition, training, bn_layer, gamma_layer, beta_layer):
        """Custom conditional batch normalization using predefined layers"""
        # Apply predefined batch normalization
        x = bn_layer(x, training=training)
        
        # Reshape condition for broadcasting
        condition = tf.reshape(condition, [-1, 1, 1, condition.shape[-1]])
        
        # Apply predefined gamma and beta layers
        gamma = gamma_layer(condition)
        beta = beta_layer(condition)
        
        # Apply scale and shift
        return x * (1 + gamma) + beta

    @staticmethod
    def generate_noise(batch_size, noise_dim=NOISE_DIM):
        """Generate Gaussian noise for the generator input."""
        return np.random.normal(0, 1, (batch_size, noise_dim))
    
    def get_config(self):
        config = super(Generator, self).get_config()
        config.update({
            'noise_dim': self.noise_dim,
            'num_classes': self.num_classes
        })
        return config
    
    @classmethod
    def from_config(cls, config):
        # Extract your custom parameters
        noise_dim = config.pop('noise_dim', 100)  # Default if missing
        num_classes = config.pop('num_classes', 10)  # Default if missing
        
        # Create instance with extracted parameters and remaining config
        return cls(noise_dim=noise_dim, num_classes=num_classes, **config)

### The Discriminator

In [4]:
# Discriminator Class with Label Conditioning
@register_keras_serializable()
class Discriminator(tf.keras.Model):
    def __init__(self, num_classes=10,**kwargs):
        super(Discriminator, self).__init__(**kwargs)
        self.num_classes = num_classes
        
        # Image processing
        self.conv1 = tf.keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same')
        self.leaky_relu1 = tf.keras.layers.LeakyReLU(alpha=0.2)
        self.dropout1 = tf.keras.layers.Dropout(0.3)
        
        self.conv2 = tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same')
        self.leaky_relu2 = tf.keras.layers.LeakyReLU(alpha=0.2)
        self.dropout2 = tf.keras.layers.Dropout(0.3)
        
        self.conv3 = tf.keras.layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same')
        self.leaky_relu3 = tf.keras.layers.LeakyReLU(alpha=0.2)
        self.dropout3 = tf.keras.layers.Dropout(0.3)
        self.flatten = tf.keras.layers.Flatten()
        
        # Label processing
        self.label_embedding = tf.keras.layers.Embedding(num_classes, 50)
        self.label_dense = tf.keras.layers.Dense(7*7, use_bias=False)
        
        # Output layer
        self.output_layer = tf.keras.layers.Dense(1)  # No activation - using from_logits=True

    def call(self, inputs, training=False):
        images, labels = inputs
        
        # Process images
        x = self.conv1(images)
        x = self.leaky_relu1(x)
        x = self.dropout1(x, training=training)
        
        x = self.conv2(x)
        x = self.leaky_relu2(x)
        x = self.dropout2(x, training=training)
        
        x = self.conv3(x)
        x = self.leaky_relu3(x)
        x = self.dropout3(x, training=training)
        
        x = self.flatten(x)
        
        # Process labels
        label_embedding = self.label_embedding(labels)
        label_embedding = tf.keras.layers.Flatten()(label_embedding)
        
        # Combine image features and label info
        combined = tf.concat([x, label_embedding], axis=1)
        
        # Output
        return self.output_layer(combined)
    
    def get_config(self):
        config = super(Discriminator, self).get_config()
        config.update({
            'num_classes': self.num_classes
        })
        return config
    
    @classmethod
    def from_config(cls, config):
        # Extract your custom parameters
        num_classes = config.pop('num_classes', 10)  # Default if missing
        
        # Create instance with extracted parameters and remaining config
        return cls(num_classes=num_classes, **config)


## The GAN Class With the Auxiliary Classifier

In [5]:
# Auxiliary Classifier to help with conditioning
@register_keras_serializable()
class AuxiliaryClassifier(tf.keras.Model):
    def __init__(self, num_classes=10,**kwargs):
        super(AuxiliaryClassifier, self).__init__(**kwargs)

        # Resize the input from (28, 28, 1) to (32, 32, 1)
        self.resize = tf.keras.layers.Resizing(32, 32)

        # Preprocessing layer to convert 1-channel input to 3 channels
        self.preprocess = tf.keras.layers.Lambda(lambda x: tf.tile(x, [1, 1, 1, 3]))

        self.num_classes = num_classes

        # Load MobileNetV2 with input shape (32, 32, 3)
        hidden = tf.keras.applications.MobileNetV2(
            input_shape=(32, 32, 3),  # Adjusted to 32x32 with 3 channels
            include_top=False,        # Exclude the top classification layer
            weights='imagenet',       # Use pre-trained ImageNet weights
        )

        # Build the model with resizing, preprocessing, and MobileNetV2
        self.model = tf.keras.Sequential([
            self.resize,                       # Resize to 32x32
            self.preprocess,                   # Convert (32, 32, 1) to (32, 32, 3)
            hidden,                            # MobileNetV2 base
            tf.keras.layers.GlobalAveragePooling2D(),  # Reduce spatial dimensions
            tf.keras.layers.Dense(512, activation='relu'),  # Dense layer
            tf.keras.layers.Dropout(0.5),             # Regularization
            tf.keras.layers.Dense(self.num_classes)  # Output layer
        ])

    def call(self, inputs, training=False):
        return self.model(inputs, training=training)
    
    def get_config(self):
        config = super(AuxiliaryClassifier, self).get_config()
        config.update({
            'num_classes': self.num_classes
        })
        return config
    
    @classmethod
    def from_config(cls, config):
        # Extract your custom parameters
        num_classes = config.pop('num_classes', 10)  # Default if missing
        
        # Create instance with extracted parameters and remaining config
        return cls(num_classes=num_classes, **config)

   
# Modified DCGAN Class with Auxiliary Classification Loss
@register_keras_serializable()
class DCGAN(tf.keras.Model):
    def __init__(self, noise_dim=NOISE_DIM, num_classes=10):
        super(DCGAN, self).__init__()
        self.noise_dim = noise_dim
        self.num_classes = num_classes

        self.generator = Generator(noise_dim, num_classes)
        self.discriminator = Discriminator(num_classes)

        # Add auxiliary classifier
        self.classifier = AuxiliaryClassifier(num_classes)

        # Optimizers
        self.generator_optimizer = tf.keras.optimizers.Adam(0.0002, 0.5)
        self.discriminator_optimizer = tf.keras.optimizers.Adam(0.0001, 0.5)
        self.classifier_optimizer = tf.keras.optimizers.Adam(0.0001)

    def compile(self):
        super(DCGAN, self).compile()

    def pretrain_classifier(self, images, labels, epochs=20):
        """Pre-train the classifier on real data"""
        print("Pre-training classifier...")

        # Create dataset from images and labels
        train_dataset = tf.data.Dataset.from_tensor_slices((images, labels))
        train_dataset = train_dataset.shuffle(len(images)).batch(128)

        # Loss function
        self.classifier.compile(
            optimizer=self.classifier_optimizer,
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy']
        )
        # Train the classifier
        self.classifier.fit(train_dataset, epochs=epochs)
        print("Classifier pre-training complete.")

        # print("Classifier Evaluation")
        # pred = self.classifier(x_test)
        # pred = np.argmax(pred,axis=1)
        # print(accuracy_score(y_test,pred))

    @tf.function
    def train_step(self, data, epoch, total_epochs):
        """Enhanced train step with auxiliary classification loss"""
        real_images, real_labels = data
        batch_size = tf.shape(real_images)[0]

        # Generate noise for fake images
        noise = tf.random.normal([batch_size, self.noise_dim])

        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            # Generate fake images
            generated_images = self.generator([noise, real_labels], training=True)

            # Get discriminator predictions
            real_output = self.discriminator([real_images, real_labels], training=True)
            fake_output = self.discriminator([generated_images, real_labels], training=True)

            # Get classifier predictions for generated images
            gen_class_logits = self.classifier(generated_images, training=False)

            # Calculate standard GAN losses
            gen_loss = self.generator_loss(fake_output)
            disc_loss = self.discriminator_loss(real_output, fake_output)

            # Calculate classification loss for generated images
            classification_loss = tf.keras.losses.SparseCategoricalCrossentropy(
                from_logits=True)(real_labels, gen_class_logits)

            # Add classification loss to generator loss (weighted)
            lambda_cls = 0.1 + (tf.cast(epoch, tf.float32) / total_epochs) * 0.3
            gen_total_loss = gen_loss + lambda_cls * classification_loss

        # Calculate gradients
        gradients_of_generator = gen_tape.gradient(
            gen_total_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(
            disc_loss, self.discriminator.trainable_variables)

        # Apply gradients
        self.generator_optimizer.apply_gradients(
            zip(gradients_of_generator, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(
            zip(gradients_of_discriminator, self.discriminator.trainable_variables))

        return {
            'd_loss': disc_loss,
            'g_loss': gen_loss,
            'cls_loss': classification_loss
        }

    def discriminator_loss(self, real_output, fake_output):
        # Use from_logits=True for numerical stability
        real_labels = tf.ones_like(real_output) * 0.9  # Label smoothing
        real_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(
            real_labels, real_output)
        fake_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(
            tf.zeros_like(fake_output), fake_output)
        total_loss = real_loss + fake_loss
        return total_loss

    def generator_loss(self, fake_output):
        return tf.keras.losses.BinaryCrossentropy(from_logits=True)(
            tf.ones_like(fake_output), fake_output)

    def train(self, images, labels=None, epochs=10, batch_size=128,train_classifier=True):
        """
        Train the model with separate images and labels

        Parameters:
        - images: The training images
        - labels: The corresponding class labels (required)
        - epochs: Number of training epochs
        - batch_size: Batch size for training
        """
        if labels is None:
            raise ValueError("Labels must be provided for conditional GAN training")

        # First pre-train the classifier
        if train_classifier:
            self.pretrain_classifier(images, labels)

        # Create dataset from images and labels
        train_dataset = tf.data.Dataset.from_tensor_slices((images, labels))
        train_dataset = train_dataset.shuffle(len(images)).batch(batch_size)

        for epoch in range(epochs):
            d_loss_total, g_loss_total, cls_loss_total = 0, 0, 0
            num_batches = 0

            # Iterate through dataset
            for real_images, real_labels in train_dataset:
                # Rescale images to [-1, 1] if needed
                if tf.reduce_max(real_images) > 1.0 or tf.reduce_min(real_images) < -1.0:
                    real_images = tf.clip_by_value(real_images, 0, 255) / 127.5 - 1
                elif tf.reduce_max(real_images) <= 1.0 and tf.reduce_min(real_images) >= 0:
                    real_images = real_images * 2 - 1

                # Train on this batch
                losses = self.train_step((real_images, real_labels), epoch, epochs)
                d_loss_total += losses['d_loss']
                g_loss_total += losses['g_loss']
                cls_loss_total += losses['cls_loss']
                num_batches += 1

            # Calculate average losses
            d_loss_avg = d_loss_total / num_batches
            g_loss_avg = g_loss_total / num_batches
            cls_loss_avg = cls_loss_total / num_batches

            # Print progress
            if epoch % 2 == 0 or epoch == epochs - 1:
                print(f"Epoch {epoch + 1}/{epochs}, D Loss: {d_loss_avg:.4f}, G Loss: {g_loss_avg:.4f}, Class Loss: {cls_loss_avg:.4f}")
                self.save_generated_images(epoch)
                self.evaluate_class_accuracy(epoch)

    def evaluate_class_accuracy(self, epoch, num_samples=500):
        """Evaluate how well the generated images match their class labels"""
        # Generate images for each class
        synthetic_images, synthetic_labels = self.generate_synthetic_data(num_samples)

        # Convert images to tensor
        synthetic_images = tf.convert_to_tensor(synthetic_images, dtype=tf.float32)
        synthetic_labels = tf.convert_to_tensor(synthetic_labels, dtype=tf.int32)

        # Get classifier predictions
        predictions = self.classifier(synthetic_images, training=False)
        predicted_classes = tf.argmax(predictions, axis=1).numpy()

        # Calculate accuracy
        accuracy = np.mean(predicted_classes == synthetic_labels.numpy())
        print(f"Epoch {epoch + 1} - Generated Image Class Accuracy: {accuracy:.4f}")

        # Create confusion matrix
        confusion = tf.math.confusion_matrix(
            labels=synthetic_labels,
            predictions=predicted_classes,
            num_classes=self.num_classes
        ).numpy()

        # Print per-class accuracy
        class_accuracies = np.diag(confusion) / np.sum(confusion, axis=1)
        for i, acc in enumerate(class_accuracies):
            print(f"  Class {i} accuracy: {acc:.4f}")

    

    def generate_synthetic_data(self, num_samples, num_classes=10):
        """
        Generate synthetic data with classifier verification.

        Args:
            num_samples (int): Total number of synthetic examples to generate.
            num_classes (int): Number of classes (digits 0-9).
            batch_size (int): Number of images to generate per batch.

        Returns:
            np.array: Synthetic images and their verified labels.
        """
        if num_classes is None:
            num_classes = self.num_classes

        synthetic_images, synthetic_labels = [], []
        samples_per_class = num_samples // num_classes

        for digit in range(num_classes):
            # Create noise and labels
            noise = tf.random.normal([samples_per_class, self.noise_dim])
            labels = tf.ones(samples_per_class, dtype=tf.int32) * digit

            # Generate class-specific images
            generated_imgs = self.generator([noise, labels], training=False)
            generated_imgs = (generated_imgs + 1) / 2.0  # Rescale to [0, 1]

            synthetic_images.extend(generated_imgs.numpy())
            synthetic_labels.extend([digit] * samples_per_class)

        return np.array(synthetic_images), np.array(synthetic_labels)

    def save_generated_images(self, epoch, output_dir='generated_images'):
        """Save generated images for each class for visualization."""
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # Generate one image per class in a 3x4 grid (for 10 classes)
        fig, axs = plt.subplots(3, 4, figsize=(12, 9))
        axs = axs.flatten()

        # Generate sample for each class
        for digit in range(min(10, self.num_classes)):
            noise = tf.random.normal([1, self.noise_dim])
            label = tf.constant([digit], dtype=tf.int32)
            gen_img = self.generator([noise, label], training=False)
            gen_img = (gen_img + 1) / 2.0  # Rescale to [0, 1]

            axs[digit].imshow(gen_img[0, :, :, 0], cmap='gray')
            axs[digit].set_title(f"Class {digit}")
            axs[digit].axis('off')

        # Remove any unused subplots
        for i in range(self.num_classes, len(axs)):
            fig.delaxes(axs[i])

        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f'epoch_{epoch}_classes.png'))
        plt.close()

    def _ensure_optimizers_initialized(self):
        if self.generator.trainable_variables:
            zero_grads_g = [tf.zeros_like(v) for v in self.generator.trainable_variables]
            self.generator_optimizer.apply_gradients(zip(zero_grads_g, self.generator.trainable_variables))
        if self.discriminator.trainable_variables:
            zero_grads_d = [tf.zeros_like(v) for v in self.discriminator.trainable_variables]
            self.discriminator_optimizer.apply_gradients(zip(zero_grads_d, self.discriminator.trainable_variables))
        if self.classifier.trainable_variables:
            zero_grads_c = [tf.zeros_like(v) for v in self.classifier.trainable_variables]
            self.classifier_optimizer.apply_gradients(zip(zero_grads_c, self.classifier.trainable_variables))

    def save(self, checkpoint_dir):
        self.generator.save(os.path.join(checkpoint_dir, 'generator.keras'))
        self.discriminator.save(os.path.join(checkpoint_dir, 'discriminator.keras'))
        self.classifier.save(os.path.join(checkpoint_dir, 'classifier.keras'))
        self._ensure_optimizers_initialized()
        checkpoint = tf.train.Checkpoint(
            generator_optimizer=self.generator_optimizer,
            discriminator_optimizer=self.discriminator_optimizer,
            classifier_optimizer=self.classifier_optimizer
        )
        checkpoint.save(os.path.join(checkpoint_dir, 'optimizer_checkpoint'))
        # Save optimizer states
        checkpoint_manager = tf.train.CheckpointManager(
            checkpoint, os.path.join(checkpoint_dir, 'optimizer_ckpt'), max_to_keep=1
        )
        checkpoint_manager.save()
        
        # Save additional configurations
        config = {
            'noise_dim': self.noise_dim,
            'num_classes': self.num_classes
        }
        np.save(os.path.join(checkpoint_dir, 'config.npy'), config)
        
        print(f"Model saved successfully to {checkpoint_dir}")
    
 
    @classmethod
    def load(cls, checkpoint_dir='checkpoints', custom_objects=None):
        """Load a complete GAN model from a checkpoint directory"""
        # Load configuration
        config_path = os.path.join(checkpoint_dir, 'config.npy')
        if not os.path.exists(config_path):
            raise FileNotFoundError(f"Configuration file not found at {config_path}")
        
        config = np.load(config_path, allow_pickle=True).item()
        
        # Create a new instance with the saved configuration
        gan = cls(noise_dim=config['noise_dim'], num_classes=config['num_classes'])
        
        # Initialize the models
        gan._initialize_models()
        
        # Load models
        gan.generator = tf.keras.models.load_model(
            os.path.join(checkpoint_dir, 'generator.keras'),
            custom_objects=custom_objects
        )
        
        gan.discriminator = tf.keras.models.load_model(
            os.path.join(checkpoint_dir, 'discriminator.keras'),
            custom_objects=custom_objects
        )
        
        gan.classifier = tf.keras.models.load_model(
            os.path.join(checkpoint_dir, 'classifier.keras'),
            custom_objects=custom_objects
        )
        
        # Initialize optimizers
        gan._ensure_optimizers_initialized()
        
        # Create optimizer checkpoint
        checkpoint = tf.train.Checkpoint(
            generator_optimizer=gan.generator_optimizer,
            discriminator_optimizer=gan.discriminator_optimizer,
            classifier_optimizer=gan.classifier_optimizer
        )
        
        # Find latest checkpoint
        latest_checkpoint = tf.train.latest_checkpoint(
            os.path.join(checkpoint_dir, 'optimizer_ckpt')
        )
        
        if latest_checkpoint:
            # Restore optimizer states
            checkpoint.restore(latest_checkpoint)
            print(f"Optimizer states restored from {latest_checkpoint}")
        else:
            print("Warning: No optimizer checkpoint found. Starting with fresh optimizers.")
        
        print(f"Model loaded successfully from {checkpoint_dir}")
        return gan
    
    def _initialize_models(self):
        """Initialize models by running a dummy forward pass"""
        # Create dummy data
        batch_size = 2
        dummy_images = tf.random.normal([batch_size, 28, 28, 1])
        dummy_labels = tf.random.uniform([batch_size], 0, self.num_classes, dtype=tf.int32)
        
        # Forward pass through each model to build them
        noise = tf.random.normal([batch_size, self.noise_dim])
        _ = self.generator([noise, dummy_labels], training=False)
        _ = self.discriminator([dummy_images, dummy_labels], training=False)
        _ = self.classifier(dummy_images, training=False)


## Training the the Full MNIST GAN

In [ ]:
# Initialize the DCGAN
dcgan = DCGAN()
dcgan.compile()




# Train the DCGAN
dcgan.train(x_train,y_train, EPOCHS, BATCH_SIZE)

# Save the model
dcgan.save('my_gan_checkpoint')

I0000 00:00:1745414689.357809   54824 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4610 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1660 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5
/home/youssef-abuzeid/.local/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/tmp/ipykernel_54824/512743676.py:16: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  hidden = tf.keras.applications.MobileNetV2(


NameError: name 'x_train' is not defined

## Load the 300,700,1000 MNIST Dataset

In [6]:


# Function to load images from a directory
def load_images_from_directory(directory, n_samples_per_class=None):
    x_data, y_data = [], []
    for class_label in range(10):
        class_dir = os.path.join(directory, str(class_label))
        images = [os.path.join(class_dir, f) for f in os.listdir(class_dir) if f.endswith(('.png', '.jpg'))]
        if n_samples_per_class:
            images = np.random.choice(images, n_samples_per_class, replace=False)
        for img_path in images:
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
            img = img.astype('float32') / 255.0  # Normalize to [0, 1]
            x_data.append(img)
            y_data.append(class_label)
    x_data = np.array(x_data).reshape(-1, 28, 28, 1)  # Reshape for CNN
    y_data = np.array(y_data)
    return x_data, y_data

# Load training subsets
x_train_300, y_train_300 = load_images_from_directory(train_dir, n_samples_per_class=300)  # 30 per class = 300 total
x_train_700, y_train_700 = load_images_from_directory(train_dir, n_samples_per_class=700)  # 70 per class = 700 total
x_train_1000, y_train_1000 = load_images_from_directory(train_dir, n_samples_per_class=1000)  # 100 per class = 1000 total

# Load test set (200 samples total)
x_test, y_test = load_images_from_directory(test_dir, n_samples_per_class=200)  

## Train the 300 GAN
### We will train the model for 75 and save the model if we are not satisfied with the results we load the model and continue training

In [7]:
dcgan_300 = DCGAN.load(checkpoint_dir='my_gan_checkpoint_300')
# dcgan_300.compile()
# Train the DCGAN on the 300-sample subset
dcgan_300.train(x_train_300, y_train_300, epochs=EPOCHS, batch_size=BATCH_SIZE)
# Save the model
!mkdir  my_gan_checkpoint_300
dcgan_300.save(checkpoint_dir='my_gan_checkpoint_300')
!mv generated_images generated_images_300


I0000 00:00:1745406169.347444  234251 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4610 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1660 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5
/home/youssef-abuzeid/.local/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/tmp/ipykernel_234251/512743676.py:16: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  hidden = tf.keras.applications.MobileNetV2(
I0000 00:00:1745406172.169639  234251 cuda_dnn.cc:529] Loaded cuDNN version 90300
/home/youssef-abuzeid/.local/lib/python3.10/site-packages/keras/src/layers/layer.py:407: UserWarning: `build()` was called on layer 'discriminator', however the layer does not have a `build()` method implemented and it looks like i

Optimizer states restored from my_gan_checkpoint_300/optimizer_ckpt/ckpt-2
Model loaded successfully from my_gan_checkpoint_300
Pre-training classifier...
Epoch 1/20


I0000 00:00:1745406190.053254  234515 service.cc:152] XLA service 0x7caf14003d40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745406190.053271  234515 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 Ti, Compute Capability 7.5
2025-04-23 13:03:10.692720: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


 7/24 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.9958 - loss: 0.0130 

I0000 00:00:1745406206.865011  234515 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

Epoch 63/75, D Loss: 1.1706, G Loss: 1.1601, Class Loss: 2.4127
Epoch 63 - Generated Image Class Accuracy: 0.0880
  Class 0 accuracy: 0.0000
  Class 1 accuracy: 0.0000
  Class 2 accuracy: 0.0000
  Class 3 accuracy: 0.0600
  Class 4 accuracy: 0.1000
  Class 5 accuracy: 0.0600
  Class 6 accuracy: 0.1000
  Class 7 accuracy: 0.0000
  Class 8 accuracy: 0.1600
  Class 9 accuracy: 0.4000


2025-04-23 13:13:19.012869: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 65/75, D Loss: 1.1462, G Loss: 1.1838, Class Loss: 2.3909
Epoch 65 - Generated Image Class Accuracy: 0.0680
  Class 0 accuracy: 0.0200
  Class 1 accuracy: 0.0200
  Class 2 accuracy: 0.0200
  Class 3 accuracy: 0.0200
  Class 4 accuracy: 0.0400
  Class 5 accuracy: 0.0600
  Class 6 accuracy: 0.1000
  Class 7 accuracy: 0.0000
  Class 8 accuracy: 0.1200
  Class 9 accuracy: 0.2800
Epoch 67/75, D Loss: 1.1313, G Loss: 1.2826, Class Loss: 2.3918
Epoch 67 - Generated Image Class Accuracy: 0.0940
  Class 0 accuracy: 0.0000
  Class 1 accuracy: 0.0200
  Class 2 accuracy: 0.0600
  Class 3 accuracy: 0.1200
  Class 4 accuracy: 0.1000
  Class 5 accuracy: 0.0800
  Class 6 accuracy: 0.1200
  Class 7 accuracy: 0.0000
  Class 8 accuracy: 0.0600
  Class 9 accuracy: 0.3800
Epoch 69/75, D Loss: 1.1367, G Loss: 1.2012, Class Loss: 2.3889
Epoch 69 - Generated Image Class Accuracy: 0.0940
  Class 0 accuracy: 0.0000
  Class 1 accuracy: 0.0200
  Class 2 accuracy: 0.0000
  Class 3 accuracy: 0.1400
  Class 4 

E0000 00:00:1745406851.262322  234251 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape indiscriminator_1/dropout_10_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


Epoch 71/75, D Loss: 1.1225, G Loss: 1.2332, Class Loss: 2.3689
Epoch 71 - Generated Image Class Accuracy: 0.0700
  Class 0 accuracy: 0.0200
  Class 1 accuracy: 0.0200
  Class 2 accuracy: 0.0000
  Class 3 accuracy: 0.1000
  Class 4 accuracy: 0.0600
  Class 5 accuracy: 0.1200
  Class 6 accuracy: 0.0200
  Class 7 accuracy: 0.0000
  Class 8 accuracy: 0.1000
  Class 9 accuracy: 0.2600
Epoch 73/75, D Loss: 1.1531, G Loss: 1.1795, Class Loss: 2.4174
Epoch 73 - Generated Image Class Accuracy: 0.0660
  Class 0 accuracy: 0.0000
  Class 1 accuracy: 0.0200
  Class 2 accuracy: 0.0000
  Class 3 accuracy: 0.0200
  Class 4 accuracy: 0.0800
  Class 5 accuracy: 0.1400
  Class 6 accuracy: 0.1800
  Class 7 accuracy: 0.0000
  Class 8 accuracy: 0.0200
  Class 9 accuracy: 0.2000
Epoch 75/75, D Loss: 1.1523, G Loss: 1.1958, Class Loss: 2.4286
Epoch 75 - Generated Image Class Accuracy: 0.0740
  Class 0 accuracy: 0.0200
  Class 1 accuracy: 0.0200
  Class 2 accuracy: 0.0000
  Class 3 accuracy: 0.0200
  Class 4 

## Train the 300 GAN
### We will train the model for 75 and save the model if we are not satisfied with the results we load the model and continue training

In [7]:
dcgan_700 = DCGAN.load(checkpoint_dir='my_gan_checkpoint_700')
# dcgan_700.compile()
# Train the DCGAN on the 700-sample subset
dcgan_700.train(x_train_700, y_train_700, epochs=EPOCHS, batch_size=BATCH_SIZE)
# Save the model
!mkdir  my_gan_checkpoint_700
dcgan_700.save(checkpoint_dir='my_gan_checkpoint_700')
!mv generated_images generated_images_700


I0000 00:00:1745400497.386411  188263 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4610 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1660 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5
/home/youssef-abuzeid/.local/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/tmp/ipykernel_188263/512743676.py:16: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  hidden = tf.keras.applications.MobileNetV2(
I0000 00:00:1745400499.885800  188263 cuda_dnn.cc:529] Loaded cuDNN version 90300
/home/youssef-abuzeid/.local/lib/python3.10/site-packages/keras/src/layers/layer.py:407: UserWarning: `build()` was called on layer 'discriminator', however the layer does not have a `build()` method implemented and it looks like i

Optimizer states restored from my_gan_checkpoint_700/optimizer_ckpt/ckpt-2
Model loaded successfully from my_gan_checkpoint_700
Pre-training classifier...
Epoch 1/20


I0000 00:00:1745400517.402144  188699 service.cc:152] XLA service 0x775e34002ce0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745400517.402158  188699 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 Ti, Compute Capability 7.5
2025-04-23 11:28:37.975599: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


 7/55 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9980 - loss: 0.0037     

I0000 00:00:1745400534.019486  188699 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

Epoch 61/75, D Loss: 0.7884, G Loss: 2.3699, Class Loss: 1.2211
Epoch 61 - Generated Image Class Accuracy: 0.8360
  Class 0 accuracy: 1.0000
  Class 1 accuracy: 1.0000
  Class 2 accuracy: 1.0000
  Class 3 accuracy: 0.9400
  Class 4 accuracy: 1.0000
  Class 5 accuracy: 0.5600
  Class 6 accuracy: 1.0000
  Class 7 accuracy: 1.0000
  Class 8 accuracy: 0.0000
  Class 9 accuracy: 0.8600
Epoch 63/75, D Loss: 0.8039, G Loss: 2.3340, Class Loss: 1.2781
Epoch 63 - Generated Image Class Accuracy: 0.8860
  Class 0 accuracy: 1.0000
  Class 1 accuracy: 1.0000
  Class 2 accuracy: 1.0000
  Class 3 accuracy: 0.8800
  Class 4 accuracy: 1.0000
  Class 5 accuracy: 1.0000
  Class 6 accuracy: 1.0000
  Class 7 accuracy: 1.0000
  Class 8 accuracy: 0.0000
  Class 9 accuracy: 0.9800


2025-04-23 11:42:04.029604: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
E0000 00:00:1745401334.807806  188263 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape indiscriminator_1_1/dropout_10_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


Epoch 65/75, D Loss: 0.7813, G Loss: 2.4038, Class Loss: 1.1320
Epoch 65 - Generated Image Class Accuracy: 0.9640
  Class 0 accuracy: 0.9800
  Class 1 accuracy: 1.0000
  Class 2 accuracy: 1.0000
  Class 3 accuracy: 0.8600
  Class 4 accuracy: 1.0000
  Class 5 accuracy: 1.0000
  Class 6 accuracy: 1.0000
  Class 7 accuracy: 1.0000
  Class 8 accuracy: 0.8600
  Class 9 accuracy: 0.9400
Epoch 67/75, D Loss: 0.7823, G Loss: 2.4025, Class Loss: 1.0616
Epoch 67 - Generated Image Class Accuracy: 0.8660
  Class 0 accuracy: 1.0000
  Class 1 accuracy: 1.0000
  Class 2 accuracy: 1.0000
  Class 3 accuracy: 0.7600
  Class 4 accuracy: 1.0000
  Class 5 accuracy: 1.0000
  Class 6 accuracy: 1.0000
  Class 7 accuracy: 1.0000
  Class 8 accuracy: 0.0000
  Class 9 accuracy: 0.9000
Epoch 69/75, D Loss: 0.7791, G Loss: 2.4043, Class Loss: 1.2107
Epoch 69 - Generated Image Class Accuracy: 0.8720
  Class 0 accuracy: 0.9800
  Class 1 accuracy: 1.0000
  Class 2 accuracy: 1.0000
  Class 3 accuracy: 0.8200
  Class 4 

E0000 00:00:1745401396.199613  188263 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape indiscriminator_1_1/dropout_10_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


Epoch 71/75, D Loss: 0.7771, G Loss: 2.4380, Class Loss: 1.3086
Epoch 71 - Generated Image Class Accuracy: 0.8780
  Class 0 accuracy: 1.0000
  Class 1 accuracy: 1.0000
  Class 2 accuracy: 1.0000
  Class 3 accuracy: 0.8200
  Class 4 accuracy: 1.0000
  Class 5 accuracy: 1.0000
  Class 6 accuracy: 1.0000
  Class 7 accuracy: 1.0000
  Class 8 accuracy: 0.0000
  Class 9 accuracy: 0.9600
Epoch 73/75, D Loss: 0.7648, G Loss: 2.4836, Class Loss: 1.0814
Epoch 73 - Generated Image Class Accuracy: 0.8900
  Class 0 accuracy: 1.0000
  Class 1 accuracy: 1.0000
  Class 2 accuracy: 1.0000
  Class 3 accuracy: 0.9000
  Class 4 accuracy: 1.0000
  Class 5 accuracy: 1.0000
  Class 6 accuracy: 1.0000
  Class 7 accuracy: 1.0000
  Class 8 accuracy: 0.0000
  Class 9 accuracy: 1.0000
Epoch 75/75, D Loss: 0.7830, G Loss: 2.4341, Class Loss: 1.1095
Epoch 75 - Generated Image Class Accuracy: 0.8940
  Class 0 accuracy: 1.0000
  Class 1 accuracy: 1.0000
  Class 2 accuracy: 0.9800
  Class 3 accuracy: 0.9800
  Class 4 

## Train the 1000 GAN
### We will train the model for 75 and save the model if we are not satisfied with the results we load the model and continue training

In [7]:
# dcgan_1000 = DCGAN()
# dcgan_1000.compile()
dcgan_1000 = DCGAN.load(checkpoint_dir='my_gan_checkpoint_1000')
# Train the DCGAN on the 1000-sample subset
dcgan_1000.train(x_train_1000, y_train_1000, epochs=EPOCHS, batch_size=BATCH_SIZE)
# Save the model
!mkdir  my_gan_checkpoint_1000
dcgan_1000.save(checkpoint_dir='my_gan_checkpoint_1000')
!mv generated_images generated_images_1000


I0000 00:00:1745416092.514567   67415 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4610 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1660 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5
/home/youssef-abuzeid/.local/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/tmp/ipykernel_67415/512743676.py:16: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  hidden = tf.keras.applications.MobileNetV2(
I0000 00:00:1745416095.102962   67415 cuda_dnn.cc:529] Loaded cuDNN version 90300
/home/youssef-abuzeid/.local/lib/python3.10/site-packages/keras/src/layers/layer.py:407: UserWarning: `build()` was called on layer 'discriminator', however the layer does not have a `build()` method implemented and it looks like it

Optimizer states restored from my_gan_checkpoint_1000/optimizer_ckpt/ckpt-2
Model loaded successfully from my_gan_checkpoint_1000
Pre-training classifier...
Epoch 1/20


I0000 00:00:1745416112.734799   67952 service.cc:152] XLA service 0x7d19240033f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745416112.734825   67952 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 Ti, Compute Capability 7.5
2025-04-23 15:48:33.324260: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


 7/79 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.9994 - loss: 0.0021     

I0000 00:00:1745416129.480761   67952 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

Epoch 65/75, D Loss: 0.7446, G Loss: 2.6008, Class Loss: 2.0032
Epoch 65 - Generated Image Class Accuracy: 0.9060
  Class 0 accuracy: 1.0000
  Class 1 accuracy: 0.9800
  Class 2 accuracy: 1.0000
  Class 3 accuracy: 0.9200
  Class 4 accuracy: 1.0000
  Class 5 accuracy: 0.9200
  Class 6 accuracy: 1.0000
  Class 7 accuracy: 1.0000
  Class 8 accuracy: 0.2400
  Class 9 accuracy: 1.0000


E0000 00:00:1745417007.745468   67415 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape indiscriminator_1_1/dropout_10_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


Epoch 67/75, D Loss: 0.7533, G Loss: 2.5585, Class Loss: 1.9516
Epoch 67 - Generated Image Class Accuracy: 0.8740
  Class 0 accuracy: 1.0000
  Class 1 accuracy: 1.0000
  Class 2 accuracy: 0.9400
  Class 3 accuracy: 0.9000
  Class 4 accuracy: 1.0000
  Class 5 accuracy: 0.9200
  Class 6 accuracy: 1.0000
  Class 7 accuracy: 0.9800
  Class 8 accuracy: 0.0000
  Class 9 accuracy: 1.0000
Epoch 69/75, D Loss: 0.7195, G Loss: 2.6771, Class Loss: 1.8908
Epoch 69 - Generated Image Class Accuracy: 0.8860
  Class 0 accuracy: 1.0000
  Class 1 accuracy: 1.0000
  Class 2 accuracy: 0.9400
  Class 3 accuracy: 0.9600
  Class 4 accuracy: 1.0000
  Class 5 accuracy: 0.9400
  Class 6 accuracy: 1.0000
  Class 7 accuracy: 1.0000
  Class 8 accuracy: 0.0200
  Class 9 accuracy: 1.0000
Epoch 71/75, D Loss: 0.7401, G Loss: 2.6328, Class Loss: 1.9177
Epoch 71 - Generated Image Class Accuracy: 0.8900
  Class 0 accuracy: 1.0000
  Class 1 accuracy: 1.0000
  Class 2 accuracy: 0.9800
  Class 3 accuracy: 1.0000
  Class 4 

E0000 00:00:1745417068.647211   67415 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape indiscriminator_1_1/dropout_10_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


Epoch 73/75, D Loss: 0.7462, G Loss: 2.6166, Class Loss: 2.0210
Epoch 73 - Generated Image Class Accuracy: 0.9660
  Class 0 accuracy: 1.0000
  Class 1 accuracy: 1.0000
  Class 2 accuracy: 0.9800
  Class 3 accuracy: 0.9600
  Class 4 accuracy: 1.0000
  Class 5 accuracy: 0.9400
  Class 6 accuracy: 1.0000
  Class 7 accuracy: 1.0000
  Class 8 accuracy: 0.7800
  Class 9 accuracy: 1.0000
Epoch 75/75, D Loss: 0.7354, G Loss: 2.6224, Class Loss: 1.9053
Epoch 75 - Generated Image Class Accuracy: 0.8960
  Class 0 accuracy: 1.0000
  Class 1 accuracy: 1.0000
  Class 2 accuracy: 0.9800
  Class 3 accuracy: 0.9800
  Class 4 accuracy: 1.0000
  Class 5 accuracy: 0.9200
  Class 6 accuracy: 1.0000
  Class 7 accuracy: 1.0000
  Class 8 accuracy: 0.0800
  Class 9 accuracy: 1.0000
mkdir: cannot create directory ‘my_gan_checkpoint_1000’: File exists
Model saved successfully to my_gan_checkpoint_1000


## Classifer Architecture

In [ ]:
def create_cnn_model():
    """Create a simple CNN model for digit classification"""
    model = tf.keras.Sequential([
        tf.keras.layers.Reshape((28, 28, 1), input_shape=(28, 28)),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

## Pipeline for testing the generated data

In [9]:
def log_message(message):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{timestamp}] {message}")

# Step 1: Load GAN models
log_message("Loading pre-trained GAN models")
dcgan_300 = DCGAN.load(checkpoint_dir='my_gan_checkpoint_300')  # Trained on 300 real samples per class
dcgan_700 = DCGAN.load(checkpoint_dir='my_gan_checkpoint_700')  # Trained on 700 real samples per class
dcgan_1000 = DCGAN.load(checkpoint_dir='my_gan_checkpoint_1000')  # Trained on 1000 real samples per class
dcgan_full = DCGAN.load(checkpoint_dir='my_gan_checkpoint')  # Trained on full MNIST dataset (best quality)

# Step 2: Load real and test data
log_message("Starting experiment pipeline for Problem 3 (GAN Evaluation)")
log_message("Loading real training data")
num_classes = 10
x_train_300, y_train_300 = load_images_from_directory(train_dir, n_samples_per_class=300)  # 300 per class (3000 total)
x_train_700, y_train_700 = load_images_from_directory(train_dir, n_samples_per_class=700)  # 700 per class (7000 total)
x_train_1000, y_train_1000 = load_images_from_directory(train_dir, n_samples_per_class=1000)  # 1000 per class (10000 total)
log_message(f"Training data shapes: 300: {x_train_300.shape}, 700: {x_train_700.shape}, 1000: {x_train_1000.shape}")

log_message("Loading test data")
x_test, y_test = load_images_from_directory(test_dir, n_samples_per_class=200)  # 200 per class (2000 total)
log_message(f"Test data shape: x_test={x_test.shape}, y_test={y_test.shape}")

# Step 3: Define the results table
# Structure: {real_samples: {generated_samples_per_class: accuracy}}
results_table = {
    300: {0: 0, 1000: {}, 2000: {}, 3000: {}},
    700: {0: 0, 1000: {}, 2000: {}, 3000: {}},
    1000: {0: 0, 1000: {}, 2000: {}, 3000: {}}
}
# Sub-dictionary for 1000, 2000, 3000 generated samples per class to store results for specific GANs
for real_samples in [300, 700, 1000]:
    for gen_samples in [1000, 2000, 3000]:
        # For each real sample size, use the corresponding GAN and dcgan_full
        if real_samples == 300:
            results_table[real_samples][gen_samples] = {'dcgan_300': 0, 'dcgan_full': 0}
        elif real_samples == 700:
            results_table[real_samples][gen_samples] = {'dcgan_700': 0, 'dcgan_full': 0}
        else:  # real_samples == 1000
            results_table[real_samples][gen_samples] = {'dcgan_1000': 0, 'dcgan_full': 0}

# Step 4: Run experiments for each combination
for real_samples, x_train, y_train in [(300, x_train_300, y_train_300),
                                       (700, x_train_700, y_train_700),
                                       (1000, x_train_1000, y_train_1000)]:
    log_message(f"Starting experiments for {real_samples} real samples per class")
    
    # Step 4.1: Evaluate with only real data (0 generated samples)
    log_message(f"Experiment: {real_samples} real + 0 generated")
    try:
        model = create_cnn_model()
        log_message("Training model on real data only")
        history = model.fit(
            x_train, y_train,
            epochs=20, batch_size=32, verbose=0
        )
        
        # Evaluate on the test set
        log_message("Evaluating model on test set")
        pred = model.predict(x_test)
        y_pred = np.argmax(pred, axis=1)
        test_accuracy = accuracy_score(y_test, y_pred)
        results_table[real_samples][0] = test_accuracy
        log_message(f"Test Accuracy (0 generated): {test_accuracy:.4f}")
    except Exception as e:
        log_message(f"Error during training/evaluation (0 generated): {str(e)}")
        continue
    
    # Step 4.2: Evaluate with 1000, 2000, 3000 synthetic samples per class using specific GANs
    for gen_samples_per_class in [1000, 2000, 3000]:
        total_gen_samples = gen_samples_per_class * num_classes  # Total synthetic samples
        log_message(f"Experiment: {real_samples} real + {gen_samples_per_class} generated per class ({total_gen_samples} total)")
        
        # Select GAN models based on real sample size
        if real_samples == 300:
            gan_models = [('dcgan_300', dcgan_300), ('dcgan_full', dcgan_full)]
        elif real_samples == 700:
            gan_models = [('dcgan_700', dcgan_700), ('dcgan_full', dcgan_full)]
        else:  # real_samples == 1000
            gan_models = [('dcgan_1000', dcgan_1000), ('dcgan_full', dcgan_full)]
        
        for gan_name, gan_model in gan_models:
            log_message(f"Using {gan_name} to generate {gen_samples_per_class} samples per class")
            
            # Generate synthetic data
            try:
                synthetic_images, synthetic_labels = gan_model.generate_synthetic_data(total_gen_samples, num_classes)
                # Combine real and synthetic data
                x_train_combined = np.concatenate((x_train, synthetic_images), axis=0)
                y_train_combined = np.concatenate((y_train, synthetic_labels), axis=0)
                log_message(f"Combined data shape: {x_train_combined.shape}")
            except Exception as e:
                log_message(f"Error during synthetic data generation with {gan_name}: {str(e)}")
                continue
            
            # Train the model
            try:
                model = create_cnn_model()
                log_message("Training model on combined real and synthetic data")
                history = model.fit(
                    x_train_combined, y_train_combined,
                    epochs=20, batch_size=32, verbose=0
                )
            except Exception as e:
                log_message(f"Error during training with {gan_name}: {str(e)}")
                continue
            
            # Evaluate on the test set
            log_message("Evaluating model on test set")
            try:
                pred = model.predict(x_test)
                y_pred = np.argmax(pred, axis=1)
                test_accuracy = accuracy_score(y_test, y_pred)
                results_table[real_samples][gen_samples_per_class][gan_name] = test_accuracy
                log_message(f"Test Accuracy with {gan_name}: {test_accuracy:.4f}")
            except Exception as e:
                log_message(f"Error during evaluation with {gan_name}: {str(e)}")
                continue

# Step 5: Display the results table
log_message("Test Accuracies Table:")
# Main table header
print("| Real Samples | 0 Generated | 1000 Generated (Specific GAN) | 1000 Generated (GAN Full) | 2000 Generated (Specific GAN) | 2000 Generated (GAN Full) | 3000 Generated (Specific GAN) | 3000 Generated (GAN Full) |")
print("|--------------|-------------|-------------------------------|---------------------------|-------------------------------|---------------------------|-------------------------------|---------------------------|")
# Rows for each real sample size
for real_samples in [300, 700, 1000]:
    # Determine the specific GAN name based on real sample size
    specific_gan = 'dcgan_300' if real_samples == 300 else 'dcgan_700' if real_samples == 700 else 'dcgan_1000'
    row = f"| {real_samples:<12} | {results_table[real_samples][0]:.4f}       | " + \
          f"{results_table[real_samples][1000][specific_gan]:.4f}                     | " + \
          f"{results_table[real_samples][1000]['dcgan_full']:.4f}                 | " + \
          f"{results_table[real_samples][2000][specific_gan]:.4f}                     | " + \
          f"{results_table[real_samples][2000]['dcgan_full']:.4f}                 | " + \
          f"{results_table[real_samples][3000][specific_gan]:.4f}                     | " + \
          f"{results_table[real_samples][3000]['dcgan_full']:.4f}                 |"
    print(row)

log_message("Experiment pipeline completed")

[2025-04-24 18:27:21] Loading pre-trained GAN models


/home/youssef-abuzeid/.local/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/tmp/ipykernel_93956/512743676.py:16: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  hidden = tf.keras.applications.MobileNetV2(
/home/youssef-abuzeid/.local/lib/python3.10/site-packages/keras/src/layers/layer.py:407: UserWarning: `build()` was called on layer 'discriminator_4', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Optimizer states restored from my_gan_checkpoint_300/optimizer_ckpt/ckpt-2
Model loaded successfully from my_gan_checkpoint_300


/home/youssef-abuzeid/.local/lib/python3.10/site-packages/keras/src/layers/layer.py:407: UserWarning: `build()` was called on layer 'discriminator_5', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Optimizer states restored from my_gan_checkpoint_700/optimizer_ckpt/ckpt-2
Model loaded successfully from my_gan_checkpoint_700


...

[2025-04-24 18:39:57] Evaluating model on test set
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
[2025-04-24 18:39:58] Test Accuracy with dcgan_full: 0.9865
[2025-04-24 18:39:58] Experiment: 1000 real + 3000 generated per class (30000 total)
[2025-04-24 18:39:58] Using dcgan_1000 to generate 3000 samples per class
[2025-04-24 18:40:01] Combined data shape: (40000, 28, 28, 1)
[2025-04-24 18:40:01] Training model on combined real and synthetic data


/home/youssef-abuzeid/.local/lib/python3.10/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


[2025-04-24 18:41:06] Evaluating model on test set
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
[2025-04-24 18:41:07] Test Accuracy with dcgan_1000: 0.9895
[2025-04-24 18:41:07] Using dcgan_full to generate 3000 samples per class
[2025-04-24 18:41:10] Combined data shape: (40000, 28, 28, 1)
[2025-04-24 18:41:10] Training model on combined real and synthetic data


/home/youssef-abuzeid/.local/lib/python3.10/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


[2025-04-24 18:42:15] Evaluating model on test set
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
[2025-04-24 18:42:15] Test Accuracy with dcgan_full: 0.9865
[2025-04-24 18:42:15] Test Accuracies Table:
| Real Samples | 0 Generated | 1000 Generated (Specific GAN) | 1000 Generated (GAN Full) | 2000 Generated (Specific GAN) | 2000 Generated (GAN Full) | 3000 Generated (Specific GAN) | 3000 Generated (GAN Full) |
|--------------|-------------|-------------------------------|---------------------------|-------------------------------|---------------------------|-------------------------------|---------------------------|
| 300          | 0.9785       | 0.9485                     | 0.9655                 | 0.9400                     | 0.9800                 | 0.9110                     | 0.9735                 |
| 700          | 0.9815       | 0.9860                     | 0.9855                 | 0.9840                     | 0.9855                 | 0.9830                     | 0.9845              

## Generated Sample Accuracy

| Real Samples | 0 Generated | 1000 Generated | 2000 Generated | 3000 Generated |
|--------------|-------------|----------------|----------------|----------------|
| 300          | 97.85%      | 94.85%          | 94%         | 91.1%          |
| 700          | 98.15%      | 98.6%          | 98.4%          | 98.3%          |
| 1000         | 98.9%       | 99%         | 98.95%         | 98.95%          |


## Proposed Pipeline
### 1. Augment the 300 real samples with 1000 generated samples
### 2. Train the GAN on the augmented dataset
### 3. Generate 1000 new samples
### 4. Train the Classifier on the augmented dataset
### 5. Evaluate the Classifier on the test set


In [7]:
from augmentations import generate_dataset_with_augmentation
# Generate augmented dataset
augmented_images,augmented_labels = generate_dataset_with_augmentation(x_train_300, y_train_300,3000, 10000)

I0000 00:00:1745507013.485078   59712 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4610 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1660 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5


Class 0: found 300 images
Generated 1000/1000 augmented images for class 0
Class 1: found 300 images
Generated 1000/1000 augmented images for class 1
Class 2: found 300 images
Generated 1000/1000 augmented images for class 2
Class 3: found 300 images
Generated 1000/1000 augmented images for class 3
Class 4: found 300 images
Generated 1000/1000 augmented images for class 4
Class 5: found 300 images
Generated 1000/1000 augmented images for class 5
Class 6: found 300 images
Generated 1000/1000 augmented images for class 6
Class 7: found 300 images
Generated 1000/1000 augmented images for class 7
Class 8: found 300 images
Generated 1000/1000 augmented images for class 8
Class 9: found 300 images
Generated 1000/1000 augmented images for class 9
Total synthetic images: 10000
Final dataset: 13000 images (3000 real + 10000 synthetic)


In [ ]:
# augmented_GAN = DCGAN()
# augmented_GAN.compile()
augmented_GAN = DCGAN.load(checkpoint_dir='my_gan_checkpoint_augmented')

augmented_GAN.train(augmented_images,augmented_labels, epochs=EPOCHS, batch_size=BATCH_SIZE)

!mkdir  my_gan_checkpoint_augmented
augmented_GAN.save(checkpoint_dir='my_gan_checkpoint_augmented')

!mv generated_images generated_images_augmented



/home/youssef-abuzeid/.local/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/tmp/ipykernel_59712/512743676.py:16: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  hidden = tf.keras.applications.MobileNetV2(
I0000 00:00:1745507022.896133   59712 cuda_dnn.cc:529] Loaded cuDNN version 90300
/home/youssef-abuzeid/.local/lib/python3.10/site-packages/keras/src/layers/layer.py:407: UserWarning: `build()` was called on layer 'discriminator', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Optimizer states restored from my_gan_checkpoint_augmented/optimizer_ckpt/ckpt-2
Model loaded successfully from my_gan_checkpoint_augmented
Pre-training classifier...
Epoch 1/20


I0000 00:00:1745507039.908406   60116 service.cc:152] XLA service 0x7830c80030f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745507039.908421   60116 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 Ti, Compute Capability 7.5
2025-04-24 17:04:00.468918: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


  7/102 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9993 - loss: 0.0065

I0000 00:00:1745507056.099308   60116 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

E0000 00:00:1745508139.238518   59712 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape indiscriminator_1/dropout_10_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


Epoch 69/75, D Loss: 0.7145, G Loss: 2.6170, Class Loss: 0.8512
Epoch 69 - Generated Image Class Accuracy: 0.8500
  Class 0 accuracy: 1.0000
  Class 1 accuracy: 1.0000
  Class 2 accuracy: 0.9200
  Class 3 accuracy: 0.7400
  Class 4 accuracy: 0.9400
  Class 5 accuracy: 0.9200
  Class 6 accuracy: 1.0000
  Class 7 accuracy: 1.0000
  Class 8 accuracy: 0.0000
  Class 9 accuracy: 0.9800
Epoch 71/75, D Loss: 0.7137, G Loss: 2.6706, Class Loss: 0.7711
Epoch 71 - Generated Image Class Accuracy: 0.8660
  Class 0 accuracy: 1.0000
  Class 1 accuracy: 1.0000
  Class 2 accuracy: 0.8800
  Class 3 accuracy: 0.8000
  Class 4 accuracy: 0.9400
  Class 5 accuracy: 0.9600
  Class 6 accuracy: 1.0000
  Class 7 accuracy: 1.0000
  Class 8 accuracy: 0.0800
  Class 9 accuracy: 1.0000


E0000 00:00:1745508203.619044   59712 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape indiscriminator_1/dropout_10_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


Epoch 73/75, D Loss: 0.6986, G Loss: 2.6859, Class Loss: 0.7153
Epoch 73 - Generated Image Class Accuracy: 0.9320
  Class 0 accuracy: 1.0000
  Class 1 accuracy: 1.0000
  Class 2 accuracy: 0.8600
  Class 3 accuracy: 0.7000
  Class 4 accuracy: 0.8600
  Class 5 accuracy: 1.0000
  Class 6 accuracy: 1.0000
  Class 7 accuracy: 1.0000
  Class 8 accuracy: 0.9200
  Class 9 accuracy: 0.9800
Epoch 75/75, D Loss: 0.6978, G Loss: 2.6692, Class Loss: 0.7586
Epoch 75 - Generated Image Class Accuracy: 0.9600
  Class 0 accuracy: 1.0000
  Class 1 accuracy: 1.0000
  Class 2 accuracy: 0.8400
  Class 3 accuracy: 0.8000
  Class 4 accuracy: 0.9800
  Class 5 accuracy: 0.9800
  Class 6 accuracy: 1.0000
  Class 7 accuracy: 1.0000
  Class 8 accuracy: 1.0000
  Class 9 accuracy: 1.0000
mkdir: cannot create directory ‘my_gan_checkpoint_augmented’: File exists
Model saved successfully to my_gan_checkpoint_augmented


: 

## Testing the proposed pipeline

In [13]:

# Step 1: Load the GAN model
log_message("Loading pre-trained GAN model")
dcgan_augmented = DCGAN.load(checkpoint_dir='my_gan_checkpoint_augmented')  # Trained on 300 real + 1000 augmented per class

# Step 2: Load real and test data
log_message("Starting experiment pipeline for Problem 3 (300 Real Samples per Class)")
log_message("Loading real training data")
num_classes = 10
x_train_300, y_train_300 = load_images_from_directory(train_dir, n_samples_per_class=300)  # 300 per class (3000 total)
log_message(f"Training data shape: 300: {x_train_300.shape}")

log_message("Loading test data")
x_test, y_test = load_images_from_directory(test_dir, n_samples_per_class=200)  # 200 per class (2000 total)
log_message(f"Test data shape: x_test={x_test.shape}, y_test={y_test.shape}")

# Step 3: Augment the dataset to 1000 samples per class
log_message("Augmenting the dataset to 1000 samples per class")
try:
    x_train_augmented, y_train_augmented = generate_dataset_with_augmentation(
        x_train_300, y_train_300, 300*10,1000*10
    )  # Augment to 1000 per class (10,000 total)
    log_message(f"Augmented data shape: {x_train_augmented.shape}")
except Exception as e:
    log_message(f"Error during augmentation: {str(e)}")
    raise

# Step 4: Define the results table
# Structure: {generated_samples_per_class: accuracy}
results_table = {
    0: 0,
    1000: 0,
    2000: 0,
    3000: 0
}

# Step 5: Run experiments for each combination of synthetic samples
for gen_samples_per_class in [0, 1000, 2000, 3000]:
    total_gen_samples = gen_samples_per_class * num_classes  # Total synthetic samples
    log_message(f"Experiment: 300 real + 1000 augmented + {gen_samples_per_class} generated per class ({total_gen_samples} total)")
    
    # Step 5.1: Combine datasets
    if gen_samples_per_class == 0:
        x_train_combined = x_train_augmented
        y_train_combined = y_train_augmented
        log_message("Using real + augmented data only, no synthetic samples")
    else:
        try:
            # Generate synthetic data
            synthetic_images, synthetic_labels = dcgan_augmented.generate_synthetic_data(total_gen_samples, num_classes)
            # Combine real + augmented + synthetic data
            x_train_combined = np.concatenate((x_train_augmented, synthetic_images), axis=0)
            y_train_combined = np.concatenate((y_train_augmented, synthetic_labels), axis=0)
            log_message(f"Combined data shape: {x_train_combined.shape}")
        except Exception as e:
            log_message(f"Error during synthetic data generation: {str(e)}")
            continue
    
    # Step 5.2: Train the model
    try:
        model = create_cnn_model()
        log_message("Training model on combined data")
        history = model.fit(
            x_train_combined, y_train_combined,
            epochs=20, batch_size=32, verbose=0
        )
    except Exception as e:
        log_message(f"Error during training: {str(e)}")
        continue
    
    # Step 5.3: Evaluate on the test set
    log_message("Evaluating model on test set")
    try:
        pred = model.predict(x_test)
        y_pred = np.argmax(pred, axis=1)
        test_accuracy = accuracy_score(y_test, y_pred)
        results_table[gen_samples_per_class] = test_accuracy
        log_message(f"Test Accuracy: {test_accuracy:.4f}")
    except Exception as e:
        log_message(f"Error during evaluation: {str(e)}")
        continue

# Step 6: Display the results table
log_message("Test Accuracies Table:")
print("| Real Samples | Augmented Samples | 0 Generated | 1000 Generated | 2000 Generated | 3000 Generated |")
print("|--------------|-------------------|-------------|----------------|----------------|----------------|")
row = f"| {300:<12} | {1000:<17} | {results_table[0]:.4f}       | {results_table[1000]:.4f}         | {results_table[2000]:.4f}         | {results_table[3000]:.4f}         |"
print(row)

log_message("Experiment pipeline completed")

[2025-04-24 19:05:00] Loading pre-trained GAN model


/tmp/ipykernel_93956/512743676.py:16: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  hidden = tf.keras.applications.MobileNetV2(
/home/youssef-abuzeid/.local/lib/python3.10/site-packages/keras/src/layers/layer.py:407: UserWarning: `build()` was called on layer 'discriminator_10', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Optimizer states restored from my_gan_checkpoint_augmented/optimizer_ckpt/ckpt-2
Model loaded successfully from my_gan_checkpoint_augmented
[2025-04-24 19:05:05] Starting experiment pipeline for Problem 3 (300 Real Samples per Class)
[2025-04-24 19:05:05] Loading real training data
[2025-04-24 19:05:05] Training data shape: 300: (3000, 28, 28, 1)
[2025-04-24 19:05:05] Loading test data
[2025-04-24 19:05:05] Test data shape: x_test=(2000, 28, 28, 1), y_test=(2000,)
[2025-04-24 19:05:05] Augmenting the dataset to 1000 samples per class
Class 0: found 300 images
Generated 1000/1000 augmented images for class 0
Class 1: found 300 images
Generated 1000/1000 augmented images for class 1
Class 2: found 300 images
Generated 1000/1000 augmented images for class 2
Class 3: found 300 images
Generated 1000/1000 augmented images for class 3
Class 4: found 300 images
Generated 1000/1000 augmented images for class 4
Class 5: found 300 images
Generated 1000/1000 augmented images for class 5
Class 6: f

/home/youssef-abuzeid/.local/lib/python3.10/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


[2025-04-24 19:05:36] Evaluating model on test set
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
[2025-04-24 19:05:37] Test Accuracy: 0.9850
[2025-04-24 19:05:37] Experiment: 300 real + 1000 augmented + 1000 generated per class (10000 total)
[2025-04-24 19:05:38] Combined data shape: (23000, 28, 28, 1)
[2025-04-24 19:05:38] Training model on combined data


...

[2025-04-24 19:06:16] Evaluating model on test set
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
[2025-04-24 19:06:16] Test Accuracy: 0.9855
[2025-04-24 19:06:16] Experiment: 300 real + 1000 augmented + 2000 generated per class (20000 total)
[2025-04-24 19:06:18] Combined data shape: (33000, 28, 28, 1)
[2025-04-24 19:06:18] Training model on combined data


/home/youssef-abuzeid/.local/lib/python3.10/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


[2025-04-24 19:07:12] Evaluating model on test set
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
[2025-04-24 19:07:12] Test Accuracy: 0.9875
[2025-04-24 19:07:12] Experiment: 300 real + 1000 augmented + 3000 generated per class (30000 total)
[2025-04-24 19:07:16] Combined data shape: (43000, 28, 28, 1)
[2025-04-24 19:07:16] Training model on combined data


/home/youssef-abuzeid/.local/lib/python3.10/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


[2025-04-24 19:08:24] Evaluating model on test set
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
[2025-04-24 19:08:24] Test Accuracy: 0.9860
[2025-04-24 19:08:24] Test Accuracies Table:
| Real Samples | Augmented Samples | 0 Generated | 1000 Generated | 2000 Generated | 3000 Generated |
|--------------|-------------------|-------------|----------------|----------------|----------------|
| 300          | 1000              | 0.9850       | 0.9855         | 0.9875         | 0.9860         |
[2025-04-24 19:08:24] Experiment pipeline completed
